# How to: Find and Access EMIT Data

**Summary**  

There are currently 3 ways to find EMIT data:

1. [EarthData Search](https://search.earthdata.nasa.gov/search)
2. [NASA's CMR API](https://www.earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr)
3. [Visions Open Access Data Portal](https://earth.jpl.nasa.gov/emit/data/data-portal/coverage-and-forecasts/)

This notebook will explain how to access Earth Surface Minteral Dust Source Investigation (EMIT) data programmaticly using the [earthaccess python library](https://github.com/nsidc/earthaccess). `earthaccess` is an easy to use library that reduces finding and downloading or streaming data over https or s3 to only a few lines of code. `earthaccess` searches NASA's Common Metadata Repository (CMR), a metadata system that catalogs Earth Science data and associated metadata records, then can be used to download granules or generate lists granule search result URLs.

**Requirements:**
+ A NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download EMIT data   
+ Selected the `emit_tutorials` environment as the kernel for this notebook.
  + For instructions on setting up the environment, follow the the `setup_instructions.md` included in the `/setup/` folder of the repository.  

**Learning Objectives**  
- How to search and access EMIT data using `earthaccess`

---

Import the required packages

In [ ]:
import os
import earthaccess
import pandas as pd
import geopandas as gp
import xarray as xr
import sys
sys.path.append('../modules/')
from emit_tools import emit_xarray


## Authentication

Login to your NASA Earthdata account and create a `.netrc` file using the `login` function from the `earthaccess` library. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/home). 

In [ ]:
earthaccess.login(persist=True)

---

## Searching for Collections

If we want to see the available EMIT collections, we can 


In [ ]:
Query = earthaccess.collection_query().keyword('emit').provider('LPCLOUD')
print(f'Collections found: {Query.hits()}')

We can retrieve metadata for these collections, and then the shortnames so we can search for granules. 

In [ ]:
# Find Collections
collections = Query.fields(['ShortName']).get(4)
# Retrieve Collection Short-names
[product['short-name'] for product in [collection.summary() for collection in collections]]

If you print the `collections` object you can explore all of the json metadata.

---
## Searching for Granules

A granule can be thought of as a unique spatiotemporal grouping within a collection. To search for granules, we simply use the `search_data` function from `earthaccess` and provide the arguments for our search. Its possible to specify search products using several criteria shown in the table below:

|dataset origin and location|spatio temporal parameters|dataset metadata parameters|
|:---|:---|:---|
|archive_center|bounding_box|concept_id
|data_center|temporal|entry_title
|daac|point|keyword
|provider|polygon|version
|cloud_hosted|line|short_name

### Point Search

In this case, we specify the `shortname`, `point`, and `temporal`, as well as `count`, which limits the maximum number of results returned. 

In [ ]:
# POINT
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    point=(-62.1123,-39.89402),
    temporal=('2022-09-03','2022-09-04'),
    count=100
)

### Bounding Box Search

You can also use a bounding box to search. To do this we will first open a geojson file containing our region of interest (ROI) then simplify it to a bounding box by getting the bounds and putting them into a tuple. We will use the `total_bounds` property to get the bounding box of our ROI, and add that to a python tuple, which is the expected data type for the bounding_box parameter `earthaccess` `search_data`.

In [ ]:
geojson = gp.read_file('../../data/isla_gaviota.geojson')
geojson.geometry

In [ ]:
bbox = tuple(list(geojson.total_bounds))
bbox

Now we can search for granules using the a bounding box.

In [ ]:
# Search Example using Bounding Box
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    bounding_box=bbox,
    temporal=('2022-09-03','2022-09-04'),
    count=100
)


### Polygon Search

A polygon can also be used to search. For a simple polygon without holes we can take the geojson we opened and grab the coordinates of the exterior ring and place them in a list.

In [ ]:
polygon = list(geojson.geometry[0].exterior.coords)
polygon

With this list of coordinate pairs we can use the `polygon` parameter for our search. 
> Note that we overwrote the `results` object, because for all 3 types spatial search, the `results` are the same for this example.

In [ ]:
# Search Example using a Polygon
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    polygon=polygon,
    temporal=('2022-09-03','2022-09-04'),
    count=100
)

---

## Working with Search Results

After we've gotten results from our search using `earthaccess` we can view the results in a table and view assets for each granule in the list. 

In [ ]:
pd.json_normalize(results)

In [ ]:
results[0]

After we have our results, there are 2 ways we an work with the data:

1. Download
2. Access in place / Stream the data. 

To download the data we can simply use the download function. This will retrieve all assets associated with a granule, and is nice if you plan to work with the data in this way.

In [ ]:
# earthaccess.download(results, '../../data/')

If we want to stream the data or further filter the assets for download we want to first create a list of URLs nested by granule using list comprehesion.

In [ ]:
emit_results_urls = [granule.data_links() for granule in results]
emit_results_urls

Now we can also split these into results for specific assets or filter out an asset using the following. In this example, we only want to access or download reflectance.

In [ ]:
filtered_asset_links = []
# Pick Desired Assets (leave _ on RFL to distinguish from RFLUNC)
desired_assets = ['RFL_'] # Add more or do individually for reflectance, reflectance uncertainty, or mask
# Step through each sublist (granule) and filter based on desired assets.
for n, granule in enumerate(emit_results_urls):
    for url in granule: 
        asset_name = url.split('/')[-1]
        if any(asset in asset_name for asset in desired_assets):
            filtered_asset_links.append(url)
filtered_asset_links

After we have our filtered list, we can stream the reflectance asset or download it. Start an https session then open it to stream the data, or download to save the file.

#### Stream Data

This may take a while to load the dataset.

In [ ]:
# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
url = filtered_asset_links[0]
granule_asset_id = url.split('/')[-1]
# Define Local Filepath
fp = fs.open(url)
# Open with `emit_xarray` function
ds = emit_xarray(fp)
ds

#### Download Filtered 

In [ ]:
# Get requests https Session using Earthdata Login Info
fs = earthaccess.get_requests_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
for url in filtered_asset_links:
    granule_asset_id = url.split('/')[-1]
    # Define Local Filepath
    fp = f'../../data/{granule_asset_id}'
    # Download the Granule Asset if it doesn't exist
    if not os.path.isfile(fp):
        with fs.get(url,stream=True) as src:
            with open(fp,'wb') as dst:
                for chunk in src.iter_content(chunk_size=64*1024*1024):
                    dst.write(chunk)

---

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 07-03-2023  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I. 